# Build TF ChIP Binary Matrix

Goal: 2 binary matrices, 1 for TFs and 1 for histone modifications. Going to work on TF here and then repeat same process for histones

Need to do: 
- get TF information: 
    - see how well the symbol map works 
- collapse to gene level (if peak overlaps the region count it)
    - need bed of regions I want/which gene they are in


In [43]:
import pandas as pd
import gffutils
from gffutils import pybedtools_integration
import pybedtools
from pybedtools.featurefuncs import gff2bed

In [44]:
tf = pd.read_table('../output/chip/ALL_TF_CHIP_filtered.bed', header=None, 
                   names=['chrom','start','end','srx','score','caller'])

In [45]:
spreadsheet = pd.read_csv('../output/chip/20171103_s2cell_chip-seq.csv')
#For now we are excluding datasets with no input: 
spreadsheet = spreadsheet[spreadsheet.input != 'no input?']
antibody_table = spreadsheet[['srx','target']]

In [46]:
tf2 = tf.merge(antibody_table, on='srx',how='left')

In [47]:
tf2.head()

,chrom,start,end,srx,score,caller,target
0,chr2L,5648,5968,SRX330995,12.65807,macs2,dsx
1,chr2L,16697,16877,SRX330995,6.65504,macs2,dsx
2,chr2L,19549,19713,SRX330995,6.06347,macs2,dsx
3,chr2L,20750,21056,SRX330995,8.21085,macs2,dsx
4,chr2L,21234,21477,SRX330995,25.66628,macs2,dsx


In [48]:
len(tf2.srx.unique()), len(tf2.target.unique())

(122, 59)

### Obtain antibody fbgn: 

In [49]:
#bed file containing introns and 1 kb upstream
intslop = pybedtools.BedTool('../output/dm6_intron_sloptranscript.bed')

In [50]:
tf_intersect = pybedtools.BedTool.from_dataframe(tf2).intersect(intslop).to_dataframe()

In [51]:
tf_intersect.head()

,chrom,start,end,name,score,strand,thickStart
0,chr2L,16697,16877,SRX330995,6.65504,macs2,dsx
1,chr2L,19549,19713,SRX330995,6.06347,macs2,dsx
2,chr2L,20750,20830,SRX330995,8.21085,macs2,dsx
3,chr2L,20973,21056,SRX330995,8.21085,macs2,dsx
4,chr2L,21376,21477,SRX330995,25.66628,macs2,dsx


In [52]:
#symbol maps map gene symbol to FBgn: 
symbolmap = pd.read_table('/data/LCDB/lcdb-references/dmel/r6-11/gtf/dmel_r6-11.SYMBOL.csv', sep=',') 
symbolmap2 = pd.read_table('../data/fb_synonym.tsv', sep=' ', header=None)
symbolmap2.columns = ['gene','a','b']

In [53]:
#make copy df
tf_intersect_copy = tf_intersect.copy()

In [54]:
tf_intersect_copy.loc[tf_intersect.thickStart == 'HP1a','thickStart'] = 'Su(var)205'
tf_intersect_copy.loc[tf_intersect.thickStart == 'Hp1a','thickStart'] = 'Su(var)205'
tf_intersect_copy.loc[tf_intersect.thickStart == 'CP190','thickStart'] = 'Cp190'
tf_intersect_copy.loc[tf_intersect.thickStart == 'CG8436','thickStart'] = 'Ibf1'
tf_intersect_copy.loc[tf_intersect.thickStart == 'CG9740','thickStart'] = 'Ibf2'
tf_intersect_copy.loc[tf_intersect.thickStart == 'NSL3','thickStart'] = 'Rcd1'
tf_intersect_copy.loc[tf_intersect.thickStart == 'UTX','thickStart'] = 'Utx'
tf_intersect_copy.loc[tf_intersect.thickStart == 'LPT','thickStart'] = 'Lpt'
tf_intersect_copy.loc[tf_intersect.thickStart == 'Trr','thickStart'] = 'trr'
tf_intersect_copy.loc[tf_intersect.thickStart == 'dCAP-D3','thickStart'] = 'Cap-D3'
tf_intersect_copy.loc[tf_intersect.thickStart == 'DnaJ1','thickStart'] = 'DnaJ-1'
tf_intersect_copy.loc[tf_intersect.thickStart == 'MYST5','thickStart'] = 'CG1894'
tf_intersect_copy.loc[tf_intersect.thickStart == 'ZIPIC','thickStart'] = 'CG7928'

In [55]:
merge1 = tf_intersect_copy.merge(symbolmap, left_on='thickStart', right_on='SYMBOL', how='left')
merge2 = merge1.merge(symbolmap2, left_on='thickStart', right_on='a', how='left')[['chrom','start','end','name',
                                                                'score','strand','thickStart','ENSEMBL','gene']]

In [56]:
merge2.fillna('')
merge2['anti_FBgn']= merge2.ENSEMBL.combine_first(merge2.gene)
trim = merge2[['chrom','start','end','name','score','strand','thickStart','anti_FBgn']]

In [57]:
#drop these because they aren't fly genes (except for Ph but it wasn't specified which ph)
trim[trim.anti_FBgn.isnull()].thickStart.unique()

array(['Rpb1', 'FLAG', 'GFP', 'Ph', 'control', 'Rpb3', 'control ', 'Rbp3'], dtype=object)

In [58]:
drop_bad_antibodies = trim[~trim.anti_FBgn.isnull()]

## Filter for RNAi TFs

In [59]:
drop_bad_antibodies.head()

,chrom,start,end,name,score,strand,thickStart,anti_FBgn
0,chr2L,16697,16877,SRX330995,6.65504,macs2,dsx,FBgn0000504
1,chr2L,19549,19713,SRX330995,6.06347,macs2,dsx,FBgn0000504
2,chr2L,20750,20830,SRX330995,8.21085,macs2,dsx,FBgn0000504
3,chr2L,20973,21056,SRX330995,8.21085,macs2,dsx,FBgn0000504
4,chr2L,21376,21477,SRX330995,25.66628,macs2,dsx,FBgn0000504


In [60]:
#Remember filter for RNAi TFs 
TF_list = pd.read_table('../output/list_of_tfs.txt', header=None)
TF_list.columns=['TF']

# make dictionary of alt fbgns

fbgn = {}
with open('/data/LCDB/lcdb-references/dmel/r6-16/fb_annotation/dmel_r6-16.fb_annotation') as f:
    next(f)
    for line in f:
        split = line.split('\t')
        first = split[1]
        seconds = split[2].split(',')
        fbgn[first] = first
        for x in seconds:
            if x:
                fbgn[x] = first
                
TF_list['update'] = TF_list.TF.map(lambda x: fbgn[x])
TF_list.drop('TF', axis=1, inplace=True)

In [61]:
TF_list.shape

(488, 1)

In [62]:
merge_on_our_TFs = drop_bad_antibodies.merge(TF_list, left_on='anti_FBgn', right_on='update', how='inner')

In [63]:
merge_on_our_TFs.head()

,chrom,start,end,name,score,strand,thickStart,anti_FBgn,update
0,chr2L,6528,7316,SRX885700,62.80834,macs2,Scm,FBgn0003334,FBgn0003334
1,chr2L,7432,7528,SRX885700,8.14595,macs2,Scm,FBgn0003334,FBgn0003334
2,chr2L,8116,8192,SRX885700,8.14595,macs2,Scm,FBgn0003334,FBgn0003334
3,chr2L,9484,9612,SRX885700,4.98406,macs2,Scm,FBgn0003334,FBgn0003334
4,chr2L,66242,66317,SRX885700,2.59312,macs2,Scm,FBgn0003334,FBgn0003334


In [64]:
len(merge_on_our_TFs.thickStart.unique())

21

In [66]:
#DSX NOT IN RNAi TF SET, SEPARATE ANALYSIS IN DIFF NOTEBOOK
drop_bad_antibodies[drop_bad_antibodies.anti_FBgn == 'FBgn0000504'].to_csv('../output/chip/dsx_peaks.bed', 
                                                                           index=False,sep='\t')

## Target gene intersect: 

In [23]:
gene_info = pybedtools.BedTool('../output/chip/dmel6.12.genes.bed')

In [24]:
targene_intersect = gene_info.intersect(pybedtools.BedTool.from_dataframe(merge_on_our_TFs), 
                                        wb=True).saveas().to_dataframe()[[3,6,7,8,9,10,11,12,13]]

/data/bergeric/miniconda3/envs/s2rnai/lib/python3.6/site-packages/pybedtools/bedtool.py:3287: UserWarning: Default names for filetype bed are:
['chrom', 'start', 'end', 'name', 'score', 'strand', 'thickStart', 'thickEnd', 'itemRgb', 'blockCount', 'blockSizes', 'blockStarts']
but file has 15 fields; you can supply custom names with the `names` kwarg
  % (self.file_type, _names, self.field_count()))


In [25]:
targene_intersect.columns = ['target_gene','chrom','start','end','srx','log10qval','caller','TF','TF_fbgn']

In [26]:
targene_intersect.head()

,target_gene,chrom,start,end,srx,log10qval,caller,TF,TF_fbgn
0,FBgn0031208,chr2L,8116,8192,SRX885700,8.145950,macs2,Scm,FBgn0003334
1,FBgn0031208,chr2L,8116,8192,SRX885698,58.313850,macs2,Scm,FBgn0003334
2,FBgn0031208,chr2L,8116,8192,SRX097617,136.588380,macs2,lilli,FBgn0041111
3,FBgn0031208,chr2L,8116,8192,SRX097617,3.405688,spp,lilli,FBgn0041111
4,FBgn0031208,chr2L,8116,8192,SRX097617,3.405688,spp,lilli,FBgn0041111


## Make sure all FBgn's are updated!!! 

In [27]:
# make dictionary of alt fbgns

fbgn = {}
with open('/data/LCDB/lcdb-references/dmel/r6-16/fb_annotation/dmel_r6-16.fb_annotation') as f:
    next(f)
    for line in f:
        split = line.split('\t')
        first = split[1]
        seconds = split[2].split(',')
        fbgn[first] = first
        for x in seconds:
            if x:
                fbgn[x] = first

In [28]:
targene_intersect['update_fbgn'] = targene_intersect.TF_fbgn.map(lambda x: fbgn[x])
targene_intersect.head()

,target_gene,chrom,start,end,srx,log10qval,caller,TF,TF_fbgn,update_fbgn
0,FBgn0031208,chr2L,8116,8192,SRX885700,8.145950,macs2,Scm,FBgn0003334,FBgn0003334
1,FBgn0031208,chr2L,8116,8192,SRX885698,58.313850,macs2,Scm,FBgn0003334,FBgn0003334
2,FBgn0031208,chr2L,8116,8192,SRX097617,136.588380,macs2,lilli,FBgn0041111,FBgn0041111
3,FBgn0031208,chr2L,8116,8192,SRX097617,3.405688,spp,lilli,FBgn0041111,FBgn0041111
4,FBgn0031208,chr2L,8116,8192,SRX097617,3.405688,spp,lilli,FBgn0041111,FBgn0041111


In [29]:
targene_intersect[targene_intersect.TF_fbgn != targene_intersect.update_fbgn]

,target_gene,chrom,start,end,srx,log10qval,caller,TF,TF_fbgn,update_fbgn


## Collapse to binary: 
- New matrix w/no duplicates

In [30]:
#If peak in gene region count it as a 1 
#only need gene,TF_fbgn
binary_collapse = targene_intersect[['target_gene','update_fbgn']]

In [31]:
binary_collapse['binary'] = 1

/data/bergeric/miniconda3/envs/s2rnai/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [32]:
binary_collapse.drop_duplicates(inplace=True)

/data/bergeric/miniconda3/envs/s2rnai/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [33]:
#index can't have duplicate entries so I need to condense this information down
binary_collapse.set_index(['target_gene','update_fbgn'], inplace=True)
matrix = binary_collapse.unstack()

In [34]:
matrix.fillna(value=0).to_csv('../output/chip/tf_matrix', sep='\t')

In [35]:
matrix.head()

binary                                                  \
update_fbgn FBgn0000283 FBgn0001206 FBgn0002775 FBgn0003042 FBgn0003334   
target_gene                                                               
FBgn0000008         NaN         1.0         NaN         NaN         1.0   
FBgn0000014         1.0         NaN         NaN         1.0         1.0   
FBgn0000015         1.0         1.0         NaN         1.0         1.0   
FBgn0000017         1.0         1.0         1.0         1.0         NaN   
FBgn0000018         NaN         NaN         1.0         NaN         NaN   

                                                                         \
update_fbgn FBgn0003567 FBgn0003607 FBgn0010328 FBgn0015602 FBgn0020388   
target_gene                                                               
FBgn0000008         NaN         NaN         1.0         NaN         1.0   
FBgn0000014         1.0         NaN         NaN         NaN         NaN   
FBgn0000015         1.0         NaN         1.0         NaN         1.0   
FBgn0000017         1.0         NaN         1.0         NaN         1.0   
FBgn0000018         NaN         NaN         NaN         NaN         NaN   

                ...                                                      \
update_fbgn     ...     FBgn0033998 FBgn0034878 FBgn0037746 FBgn0038016   
target_gene     ...                                                       
FBgn0000008     ...             1.0         NaN         NaN         1.0   
FBgn0000014     ...             NaN         NaN         NaN         NaN   
FBgn0000015     ...             NaN         1.0         1.0         1.0   
FBgn0000017     ...             1.0         NaN         1.0         1.0   
FBgn0000018     ...             NaN         NaN         NaN         NaN   

                                                                         \
update_fbgn FBgn0039019 FBgn0039740 FBgn0041111 FBgn0259785 FBgn0262656   
target_gene                                                               
FBgn0000008         NaN         1.0         1.0         NaN         NaN   
FBgn0000014         NaN         1.0         1.0         NaN         NaN   
FBgn0000015         1.0         1.0         1.0         1.0         NaN   
FBgn0000017         NaN         1.0         1.0         1.0         NaN   
FBgn0000018         NaN         NaN         NaN         NaN         NaN   

                         
update_fbgn FBgn0263667  
target_gene              
FBgn0000008         NaN  
FBgn0000014         1.0  
FBgn0000015         1.0  
FBgn0000017         1.0  
FBgn0000018         NaN  

[5 rows x 21 columns]

In [36]:
matrix.sum(axis=0)

        update_fbgn
binary  FBgn0000283    5365.0
        FBgn0001206    1498.0
        FBgn0002775    5072.0
        FBgn0003042    1770.0
        FBgn0003334    2321.0
        FBgn0003567    4903.0
        FBgn0003607    1139.0
        FBgn0010328    3702.0
        FBgn0015602    1041.0
        FBgn0020388    3343.0
        FBgn0023518    1510.0
        FBgn0033998    6304.0
        FBgn0034878    1336.0
        FBgn0037746    1772.0
        FBgn0038016    5860.0
        FBgn0039019    1544.0
        FBgn0039740    3912.0
        FBgn0041111    2104.0
        FBgn0259785    5535.0
        FBgn0262656     534.0
        FBgn0263667    3459.0
dtype: float64

In [37]:
row = targene_intersect[targene_intersect.update_fbgn == 'FBgn0033998']
row.head()

,target_gene,chrom,start,end,srx,log10qval,caller,TF,TF_fbgn,update_fbgn
103,FBgn0002121,chr2L,17515,17676,SRX326966,2.69786,macs2,row,FBgn0033998,FBgn0033998
104,FBgn0002121,chr2L,17769,18025,SRX326966,50.72078,macs2,row,FBgn0033998,FBgn0033998
105,FBgn0002121,chr2L,18168,18260,SRX326966,50.72078,macs2,row,FBgn0033998,FBgn0033998
106,FBgn0002121,chr2L,20391,20830,SRX326966,31.25111,macs2,row,FBgn0033998,FBgn0033998
107,FBgn0002121,chr2L,20973,21065,SRX326966,31.25111,macs2,row,FBgn0033998,FBgn0033998


In [38]:
row.srx.unique()

array(['SRX326966'], dtype=object)

In [39]:
row.caller.unique()

array(['macs2', 'spp'], dtype=object)

In [40]:
row[['chrom','start','end']].to_csv('../output/chip/row.bed', header=None, index=False, sep='\t')